In [1]:
# 신경망은 데이터에 대한 연산을 수행하는 계층(layer)/모듈(module)로 구성되어 있습니다.

# torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공합니다.

# PyTorch의 모든 모듈은 nn.Module 의 하위 클래스(subclass) 입니다.

# 신경망은 다른 모듈(계층; layer)로 구성된 모듈입니다.

# 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있습니다.

import os
# os 모듈은 운영 체제와 상호 작용하기 위한 함수들을 제공합니다. 파일 경로 조작이나 환경 변수 등을 다루는 데 사용될 수 있습니다.

import torch
from torch import nn
#nn은 PyTorch의 신경망 모듈(nn.Module)을 정의하는 데 사용됩니다. 신경망 계층, 손실 함수 등을 포함한 다양한 구성 요소를 제공합니다.

from torch.utils.data import DataLoader
from torchvision import datasets, transforms






In [2]:
# 가능한 경우 GPU 또는 MPS와 같은 하드웨어 가속기에서 모델을 학습하려고 합니다.
# torch.cuda 또는 torch.backends.mps 가 사용 가능한지 확인해보고, 그렇지 않으면 CPU를 계속 사용합니다.

device=(
    "cuda"
    if torch.cuda.is_available()
    else"mps"
    if torch.backends.mps.is_available()
    else "cpu"

)
print(f"Using {device} device")

Using cpu device


In [10]:
# 신경망 모델을 nn.Module 의 하위클래스로 정의하고, __init__ 에서 신경망 계층들을 초기화합니다.
# nn.Module 을 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산들을 구현합니다.

class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__() # 부모 클래스인 nn.Module의 생성자를 호출하여 초기화합니다.
    self.flatten=nn.Flatten() # nn.Flatten()은 이미지 데이터의 다차원 배열을 1차원으로 평탄화하는 모듈입니다. 입력 이미지를 1차원 벡터로 변환하기 위해 사용됩니다.
    self.linear_relu_stack = nn.Sequential(

 #  nn.Sequential은 순차적으로 연결된 모듈들을 포함하는 컨테이너입니다.
 #  여기서는 선형 레이어와 ReLU 활성화 함수를 쌓아서 신경망의 중간 레이어를 정의합니다.
 #  마지막으로 출력 레이어로 10개의 클래스를 분류하기 위한 선형 레이어가 추가됩니다.

        nn.Linear(28*28,512), #첫 번째 선형 레이어입니다. 입력 크기는 28x28 (이미지의 픽셀 수), 출력 크기는 512입니다. 이 레이어는 입력과 출력 사이의 선형 변환을 수행합니다.
        nn.ReLU(), #ReLU(Rectified Linear Unit) 활성화 함수입니다. 이 함수는 비선형성을 도입하여 신경망이 복잡한 관계를 학습할 수 있도록 돕습니다.
        nn.Linear(512,512), #두 번째 선형 레이어입니다. 입력 크기는 512이고, 출력 크기도 512입니다. 이 레이어는 첫 번째 선형 레이어의 출력을 입력으로 받아 다시 선형 변환을 수행합니다.
        nn.ReLU(),
        nn.Linear(512,10), #세 번째 선형 레이어입니다. 입력 크기는 512이고, 출력 크기는 10입니다. 이 레이어는 중간 레이어의 출력을 입력으로 받아 최종적으로 10개의 클래스를 분류하기 위한 선형 변환을 수행합니다.

    )

  def forward(self, x): #forward 메서드는 신경망 모델의 순전파(forward pass) 연산을 정의합니다. 주어진 입력 데이터 x를 받아서 출력을 계산하고 반환합니다.
    x=self.flatten(x) #입력 데이터를 평탄화하는 flatten 모듈을 통과시킵니다.
    logits = self.linear_relu_stack(x) #평탄화된 입력 데이터를 중간 레이어를 통과시켜 로짓(logit) 값을 계산합니다.
    return logits #로짓 값을 반환합니다. 이는 신경망 모델의 최종 출력입니다.




In [11]:
#NeuralNetwork 의 인스턴스(instance)를 생성하고 이를 device 로 이동한 뒤, 구조(structure)를 출력합니다.

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [25]:
# 모델을 사용하기 위해 입력 데이터를 전달합니다.
# 모델에 입력을 전달하여 호출하면 2차원 텐서를 반환합니다.
# 2차원 텐서의 dim=0은 각 분류(class)에 대한 원시(raw) 예측값 10개가, dim=1에는 각 출력의 개별 값들이 해당합니다.
# 원시 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 예측 확률을 얻습니다.

X= torch.rand(1,28,28,device=device) #크기가 1x28x28인 랜덤한 값을 가지는 입력 데이터 X를 생성합니다. device는 계산을 수행하는 디바이스(CPU 또는 GPU)를 나타냅니다.
logits=model(X) #생성한 입력 데이터 X를 모델에 입력하여 로짓 값을 얻습니다. model은 앞서 정의한 신경망 모델로, 입력에 대한 로짓 값을 반환합니다.
pred_probab = nn.Softmax(dim=1)(logits) #로짓 값을 소프트맥스 함수를 사용하여 확률로 변환합니다. nn.Softmax(dim=1)은 로짓 값을 정규화하여 클래스에 속할 확률 분포를 얻기 위해 사용됩니다. dim=1은 클래스 축에 대해 소프트맥스 함수를 적용함을 의미합니다.
y_pred = pred_probab.argmax(1) #소프트맥스 함수의 출력에서 가장 큰 값을 가지는 클래스를 예측값으로 선택합니다. argmax(1)은 클래스 축(차원)에 대해 가장 큰 값을 가지는 인덱스를 반환합니다.
print(f"Predicated class : {y_pred}") #예측된 클래스를 출력합니다

Predicated class : tensor([9])


In [33]:
# FashionMNIST 모델의 계층들을 살펴보겠습니다. 이를 설명하기 위해, 28x28 크기의 이미지 3개로 구성된 미니배치를 가져와, 신경망을 통과할 때 어떤 일이 발생하는지 알아보겠습니다.

#nn.Flatten

input_image = torch.rand (3,28,28)
print (input_image.size())

flatten = nn.Flatten()
flat_image=flatten(input_image) #nn.Flatten 계층을 초기화하여 각 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환
print(flat_image.size())


torch.Size([3, 28, 28])
torch.Size([3, 784])


In [32]:
# nn.Linear

# 입력 데이터를 nn.Linear 레이어에 통과시키고, 그 결과를 출력하는 부분입니다.
# 선형 계층 은 저장된 가중치(weight)와 편향(bias)을 사용하여 입력에 선형 변환(linear transformation)을 적용하는 모듈
layer1=nn.Linear(in_features=28*28, out_features=20) #입력 특성의 개수 in_features를 28*28로 설정하고, 출력 특성의 개수 out_features를 20으로 설정하여 nn.Linear 레이어를 생성합니다.
hidden1=layer1(flat_image) #flat_image를 layer1에 통과시킵니다. 이는 입력 데이터를 선형 변환(linear transformation)하고 활성화 함수를 적용하는 과정입니다. 결과로서 hidden1에 변환된 출력이 저장됩니다.
print(hidden1.size()) #hidden1의 크기를 출력합니다. 이는 변환된 출력의 크기(shape)를 나타냅니다.


torch.Size([3, 20])


In [31]:
# nn.ReLU

#비선형 활성화(activation)는 모델의 입력과 출력 사이에 복잡한 관계(mapping)를 만듭니다.
#비선형 활성화는 선형 변환 후에 적용되어 비선형성(nonlinearity) 을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 돕습니다.

print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[0.0000, 0.0000, 0.3545, 0.1886, 0.2253, 0.0000, 0.0650, 0.0000, 0.0000,
         0.0000, 0.0720, 0.0747, 0.0000, 0.0000, 0.0000, 0.0000, 0.0722, 0.1104,
         0.6081, 0.0871],
        [0.0000, 0.0000, 0.4013, 0.2477, 0.1357, 0.0995, 0.0758, 0.0000, 0.0232,
         0.0621, 0.0000, 0.1698, 0.0000, 0.0000, 0.0000, 0.0000, 0.3417, 0.0000,
         0.9319, 0.3432],
        [0.0000, 0.0000, 0.3554, 0.2141, 0.2346, 0.0000, 0.2972, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.6090, 0.2232]], grad_fn=<ReluBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.3545, 0.1886, 0.2253, 0.0000, 0.0650, 0.0000, 0.0000,
         0.0000, 0.0720, 0.0747, 0.0000, 0.0000, 0.0000, 0.0000, 0.0722, 0.1104,
         0.6081, 0.0871],
        [0.0000, 0.0000, 0.4013, 0.2477, 0.1357, 0.0995, 0.0758, 0.0000, 0.0232,
         0.0621, 0.0000, 0.1698, 0.0000, 0.0000, 0.0000, 0.0000, 0.3417, 0.0000,
         0.9319, 0.3432],
       

In [35]:
# nn.Sequential

# nn.Sequential 은 순서를 갖는 모듈의 컨테이너입니다. 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달됩니다.
# 순차 컨테이너(sequential container)를 사용하여 아래의 seq_modules 와 같은 신경망을 빠르게 만들 수 있습니다.

seq_modules=nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)

)
input_image=torch.rand(3,28,28)
logits=seq_modules(input_image)

In [36]:
# nn.Softmax

# 신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달될 ([-infty, infty] 범위의 원시 값(raw value)인) logits 를 반환합니다.
# logits는 모델의 각 분류(class)에 대한 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 조정(scale)됩니다. dim 매개변수는 값의 합이 1이 되는 차원을 나타냅니다.

softmax=nn.Softmax(dim=1)
pred_probab=softmax(logits)

In [38]:
# 모델 매개변수

# 신경망 내부의 많은 계층들은 매개변수화(parameterize) 됩니다. 즉, 학습 중에 최적화되는 가중치와 편향과 연관지어집니다.
# nn.Module 을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적(track)되며, 모델의 parameters() 및 named_parameters() 메소드로 모든 매개변수에 접근할 수 있게 됩니다.

# 이 예제에서는 각 매개변수들을 순회하며(iterate), 매개변수의 크기와 값을 출력합니다.

print(f'Model structure : {model}\n\n')

for name, param in model.named_parameters():
  print(f"Layer: {name} | Size : {param.size()} | Values : {param[:2]}\n")

Model structure : NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size : torch.Size([512, 784]) | Values : tensor([[ 0.0232, -0.0237, -0.0311,  ..., -0.0289,  0.0066,  0.0168],
        [-0.0116, -0.0313, -0.0206,  ...,  0.0256, -0.0200,  0.0008]],
       grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias | Size : torch.Size([512]) | Values : tensor([-0.0315,  0.0349], grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight | Size : torch.Size([512, 512]) | Values : tensor([[ 0.0189, -0.0330, -0.0045,  ..., -0.0283, -0.0289, -0.0353],
        [ 0.0213, -0.0080,  0.0263,  ...,  0.0268, -0.0104, -0.0023]],
       grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.bias |